In [ ]:
%cd /ibex/user/slimhy/PADS/code
import os
import numpy as np
from util.misc import load_pickle, get_bb_vecs



def radial_sort(points):
    """
    Radially sort points in a NumPy array.
    
    :param points: NumPy array of shape (n, 2) where n is the number of points
    :return: NumPy array of the same shape, with points sorted radially
    """
    # Calculate the centroid
    centroid = np.mean(points, axis=0)
    
    # Calculate the angles
    angles = np.arctan2(points[:, 1] - centroid[1], points[:, 0] - centroid[0])
    
    # Sort the points based on their angles
    sorted_indices = np.argsort(angles)
    
    return points[sorted_indices]

    
def get_part_labels(bb):
    return np.array([b[0][1] for b in bb])

def get_part_bbs__(bb):
    all_bbs = []
    for bb_label, bb_prim in bb:
        centroid, vecs = get_bb_vecs(bb_prim)
        stacked_data = np.vstack((centroid, vecs))
        all_bbs.append(stacked_data)
    return np.array(all_bbs)

def get_part_bbs(bb):
    try:
        return 1, get_part_bbs__(bb)
    except:
        return 0, bb

/ibex/user/slimhy/PADS/code


In [2]:
from tqdm.notebook import tqdm

latents_dir = os.path.join("/ibex/project/c2273/PADS/3DCoMPaT", "latents")
bbs_dir = os.path.join("/ibex/project/c2273/PADS/3DCoMPaT", "bounding_boxes")
bb_data_dir = "/ibex/project/c2273/3DCoMPaT/packaged"

for f in tqdm(sorted(os.listdir(latents_dir))):
    if f.endswith(".npy"):
        bb_file = f.replace(".npy", "_part_bbs.npy")
        label_file = f.replace(".npy", "_part_labels.npy")
        
        bb_data_f = os.path.join(bb_data_dir, f.replace(".npy", "_bbs.pkl"))
        bb_data = load_pickle(os.path.join(bbs_dir, bb_data_f))

        bb_labels = get_part_labels(bb_data)
        err, bb_coords = get_part_bbs(bb_data)
        
        if err == 0:
            break
        # Write to bb dir
        np.save(os.path.join(bbs_dir, bb_file), bb_coords)
        np.save(os.path.join(bbs_dir, label_file), bb_labels)

In [4]:
import os
import numpy as np
from tqdm.notebook import tqdm
from multiprocessing import Pool, cpu_count
from functools import partial


def process_file(f, latents_dir, bbs_dir, bb_data_dir):
    if f.endswith(".npy"):
        bb_file = f.replace(".npy", "_part_bbs.npy")
        label_file = f.replace(".npy", "_part_labels.npy")
        
        bb_data_f = os.path.join(bb_data_dir, f.replace(".npy", "_bbs.pkl"))
        bb_data = load_pickle(os.path.join(bbs_dir, bb_data_f))

        bb_labels = get_part_labels(bb_data)
        err, bb_coords = get_part_bbs(bb_data)
        
        if err == 0:
            return 0
        
        # Write to bb dir
        np.save(os.path.join(bbs_dir, bb_file), bb_coords)
        np.save(os.path.join(bbs_dir, label_file), bb_labels)
    
    return 1

latents_dir = os.path.join("/ibex/project/c2273/PADS/3DCoMPaT", "latents")
bbs_dir = os.path.join("/ibex/project/c2273/PADS/3DCoMPaT", "bounding_boxes")
bb_data_dir = "/ibex/project/c2273/3DCoMPaT/packaged"

files = sorted([f for f in os.listdir(latents_dir) if f.endswith(".npy")])

# Use all available CPU cores
num_processes = cpu_count() // 2

# Create a partial function with fixed arguments
process_file_partial = partial(process_file, latents_dir=latents_dir, bbs_dir=bbs_dir, bb_data_dir=bb_data_dir)

with Pool(processes=num_processes) as pool:
    results = list(tqdm(pool.imap(process_file_partial, files), total=len(files)))

if 0 in results:
    print("Processing stopped due to an error (err == 0)")
else:
    print("All files processed successfully")


  0%|          | 0/261545 [00:00<?, ?it/s]

All files processed successfully
